In [ ]:
!git clone https://github.com/gprs1809/ML-for-Good-Hackathon.git

Cloning into 'ML-for-Good-Hackathon'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 62 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [ ]:
!pip install textract

In [ ]:
!pip install rake-nltk

     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 749 kB 69.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import pandas as pd
import os
import textract
import re
import gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
for a,b,c in os.walk('/content/ML-for-Good-Hackathon/Data'):
  print(a,b,c)

/content/ML-for-Good-Hackathon/Data ['ProlificAcademic', 'CrisisLogger', 'FocusGroups'] []
/content/ML-for-Good-Hackathon/Data/ProlificAcademic ['April 2021', 'April 2020', 'November 2020', 'May 2020'] ['_CRISIS_Data_Release_1.0.docx']
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2021 ['Data', 'Dictionaries'] []
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2021/Data [] ['CRISIS_Parent_April_2021.csv', 'CRISIS_Adult_April_2021.csv']
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2021/Dictionaries [] ['CRISISPAAdultApril2021_DataDictionary_2021-11-01.csv', 'CRISISPAParentApril2021_DataDictionary_2021-11-01.csv']
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2020 ['Data', 'Dictionaries'] []
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2020/Data [] ['CRISIS_Parent_April_2020.csv', 'CRISIS_Adult_April_2020.csv']
/content/ML-for-Good-Hackathon/Data/ProlificAcademic/April 2020/Dictionaries [] ['CRISIS_April_Recoded_Variables_D

In [ ]:
#function for preprocessing of word documents comprising of dialogues between Moderators, Parents and Administrator
def doc_preprocessing(path):
  text = textract.process(path)
  text = text.decode('utf-8')
  text =text.replace('\n',"")
  text = text.replace('\t',"")
  text = text.replace('(silence)',"")
  text = text.replace('. ','.')
  split_sent=re.split(r'[?.]', text)
  split_sent = list(filter(('').__ne__, split_sent))
  dial_characters = ('Administrator','Moderator','Parent','Speaker',' Administrator',' Moderator',' Parent',' Speaker')
  dial_start=[ind for ind,l in enumerate(split_sent) if l.startswith(dial_characters)]
  list_of_dial = []
  for i in range(len(dial_start)-1):
    list_of_dial.append('.'.join(split_sent[dial_start[i]:dial_start[i+1]]))
  list_of_Parent_dial = [sent for sent in list_of_dial if sent.startswith('Parent')]
  num_ind = [sent.split(" ")[1].split(':')[0] for sent in list_of_Parent_dial]
  parent_num = list(set([int(word) for sent in num_ind for word in sent.split() if word.isdigit()]))
  all_dial_comb = []
  for i in parent_num:
        all_dial_comb.append('.'.join([sent for sent in list_of_Parent_dial if sent.startswith(('Parent '+str(i)+':',' Parent '+str(i)+':'))]))
  all_d_comb_n=[]
  for i in range(len(all_dial_comb)):
    all_d_comb_n.append(all_dial_comb[i].replace('Parent '+str(i+1)+":","").replace(' Parent '+str(i+1)+":",""))
  dial_dict = dict(zip(['Parent '+str(i) for i in parent_num],all_d_comb_n))
  return dial_dict

In [106]:
# we can draw insights from the extracted summaries of dialogues. Either this could be our result or we can work on these summaries further
#for say, sentiment analysis or clustering.
def extractive_text_summarization(path):
  imp_dial_parents = doc_preprocessing(path)
  # Summary (0.5% of the original content).
  summ_per = [summarize(dialogue.replace(".",". "), ratio = 0.1) for dialogue in list(imp_dial_parents.values())]
  # Summary (200 words)
  summ_words = [summarize(dialogue.replace(".",". "), word_count = 200) for dialogue in list(imp_dial_parents.values())]
  keys = list(imp_dial_parents.keys())
  summ_per = dict(zip(keys,summ_per))
  summ_words = dict(zip(keys,summ_words))
  r = Rake()
  key_ph_list = []
  for i in range(len(keys)):
    r.extract_keywords_from_text(list(imp_dial_parents.values())[i])
    key_ph_list.append(r.get_ranked_phrases()[:30])
  key_phrases = dict(zip(keys,key_ph_list))
  return summ_per, summ_words, key_phrases

In [107]:
list_of_docs = ['./ML-for-Good-Hackathon/Data/FocusGroups/'+file for file in os.listdir('./ML-for-Good-Hackathon/Data/FocusGroups')]

In [108]:
list_of_docs

['./ML-for-Good-Hackathon/Data/FocusGroups/Media_Group2.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Gaming_Group2.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Media_Group3.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Gaming_Group1.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Social_Group1.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Social_Group3.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Social_Group4.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Gaming_Group3.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Gaming_Group4.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Media_Group4.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Media_Group1.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/LowPIU_Group2.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/LowPIU_Group1.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/LowPIU_Group3.docx',
 './ML-for-Good-Hackathon/Data/FocusGroups/Social_Group2.docx']

In [109]:
doc_preprocessing(list_of_docs[3])

{'Parent 1': 'Hi.Hi everyone.My name is Parent 1, I have two boys, a 8-year-old and almost 3-year-old.I think the most recent show that I personally watched with no children around was Schitt\'s Creek, which was amazing and helped me get through all these difficult times lately.I was a little bit late to the party, I started watching it maybe a month or two ago, but I could not stop.Even if the work hours were late, I had to make sure I\'ll catch an episode before I go sleep.With the kids, well with the older kid, I think a few months ago we watched Floor Is Lava on Netflix, which was fun to watch as a family.Besides that, I think my son watches lots of Marvel type things with my husband, which I don\'t participate in.Yeah, so I have a similar experience with my eight-year-old.Pre-pandemic he was still in second grade so the use of computers was very limited.Maybe once a week he would type, words of the week was one of the options I picked to do.That was his only interaction with compu

In [110]:
doc_preprocessing(list_of_docs[5])

{'Parent 1': '.Hi.Hi.I\'m like, "Why don\'t I see someone else."Parent 2:Hi, I\'m [Parent 2 00:09:11].Hi, I\'m Parent 1.I guess we\'re the only two here.Oh, right, right, right.Hi.Hello.I\'m going to get something to put the phone in.Hi.Hi, my name is Parent 1.I am the mom of three.I have my oldest son is 26.I have a 24 year old girl and then my youngest one is 16, Parent 1, Child 1.We have to write it.I have to get up for a second, get a pen.Okay, I\'ll go.Oh, I\'m sorry.I think.All right, so for my 16 year old, before the pandemic, we had a set sleeping schedule.His grades were good, but there was a set sleeping schedule.During the pandemic and after the pandemic, that basically was interrupted and I was trying to over compensate for the amount of time being in the house and the lack of socialization with his friends, so the sleeping time was interrupted and it has continued.At this point, basically he goes to sleep whenever he\'s really tired.One thing, that was one of the negative 

In [118]:
extractive_text_summarization(list_of_docs[5])

({'Parent 1': 'The time spent in the room, it has come to the point that he\'s basically having all meals in the room because either he only has a certain amount of minutes, like three minutes, according to him, to have breakfast before he starts class.\nI mean, it\'s a period that has affected everyone and at home, us parents, we have tried, like in my case, I\'ve tried to, like I said before, overcompensate with giving him the free time to be online for as long as he wants, staying up until whenever he gets tired.\n" That has created a problem, but I think in every household, we as parents, we try to overcompensate the fact that they were not interacting with their friends, they\'re not having physical activity, they\'re not outside, so we just let them get away with basically just being online or playing games.\nWhen they are together physically, I know for my son, they become more introverted because they\'re so used to talking to each other through a screen, when they\'re in perso

In [111]:
imp_dial_parents = doc_preprocessing(list_of_docs[5])
# Summary (0.5% of the original content).
#summ_per = [summarize(dialogue, ratio = 0.1) for dialogue in list(imp_dial_parents.values())]
# Summary (200 words)
#summ_words = [summarize(dialogue, word_count = 200) for dialogue in list(imp_dial_parents.values())]

In [ ]:
list(imp_dial_parents.values())

['.Hi.Hi.I\'m like, "Why don\'t I see someone else."Parent 2:Hi, I\'m [Parent 2 00:09:11].Hi, I\'m Parent 1.I guess we\'re the only two here.Oh, right, right, right.Hi.Hello.I\'m going to get something to put the phone in.Hi.Hi, my name is Parent 1.I am the mom of three.I have my oldest son is 26.I have a 24 year old girl and then my youngest one is 16, Parent 1, Child 1.We have to write it.I have to get up for a second, get a pen.Okay, I\'ll go.Oh, I\'m sorry.I think.All right, so for my 16 year old, before the pandemic, we had a set sleeping schedule.His grades were good, but there was a set sleeping schedule.During the pandemic and after the pandemic, that basically was interrupted and I was trying to over compensate for the amount of time being in the house and the lack of socialization with his friends, so the sleeping time was interrupted and it has continued.At this point, basically he goes to sleep whenever he\'s really tired.One thing, that was one of the negative things that 

In [ ]:
len(list(imp_dial_parents.values()))

5

In [100]:
a =list(imp_dial_parents.values())[1]

In [103]:
a=a.replace(".",". ")

In [105]:
summarize(a, word_count=100)

'I got to the point where all I did all day long was scream and scream and scream that I just personally didn\'t like being in my house myself, so I could only imagine what my kids were going through.\nShe\'s like, "Okay, Parent 2, Friend 1 told me she\'s busy today and she forgot.\nIf Parent 2, Friend 1 goes here, I know she\'s with Parent 2, Friend 3, which means she lied to me.\n" But I\'m like but the other day, her friend was asking her, "Oh, you went to a picnic with these girls.'

In [ ]:
list(imp_dial_parents.values())[0]

'.Hi.Hi.I\'m like, "Why don\'t I see someone else."Parent 2:Hi, I\'m [Parent 2 00:09:11].Hi, I\'m Parent 1.I guess we\'re the only two here.Oh, right, right, right.Hi.Hello.I\'m going to get something to put the phone in.Hi.Hi, my name is Parent 1.I am the mom of three.I have my oldest son is 26.I have a 24 year old girl and then my youngest one is 16, Parent 1, Child 1.We have to write it.I have to get up for a second, get a pen.Okay, I\'ll go.Oh, I\'m sorry.I think.All right, so for my 16 year old, before the pandemic, we had a set sleeping schedule.His grades were good, but there was a set sleeping schedule.During the pandemic and after the pandemic, that basically was interrupted and I was trying to over compensate for the amount of time being in the house and the lack of socialization with his friends, so the sleeping time was interrupted and it has continued.At this point, basically he goes to sleep whenever he\'s really tired.One thing, that was one of the negative things that I

In [ ]:
from rake_nltk import Rake

import nltk
nltk.download('stopwords')
nltk.download('punkt')
r = Rake()

# Extraction given the text.
r.extract_keywords_from_text(list(res.values())[0])

# Extraction given the list of strings where each string is a sentence.
#r.extract_keywords_from_sentences(all_d_comb_n)

# To get keyword phrases ranked highest to lowest.
r.get_ranked_phrases()[:30]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['getting together socially outside yet',
 'special ed teachers would come',
 'play dates yet anymore',
 'video game addiction thing',
 'got special ed teachers',
 'every single night playing',
 'makes things taste good',
 'general ed teachers',
 'general ed teachers',
 'child mind institute',
 'general ed teacher',
 'parent 2 mentioned',
 'little tiny clips',
 'wilson book programs',
 'need outside help',
 'parent 3 :.',
 'would love help',
 'moved far away',
 'extra screen time',
 'extra screen time',
 '10 minutes ."',
 'whole new life',
 'missing entire homeworks',
 'totally fine ."',
 'talk every day',
 'really helped facilitate',
 'totally different era',
 'one negative thing',
 'could easily take',
 'would never lie']

In [ ]:
res = doc_preprocessing(list_of_docs[7])

In [ ]:
res

{'Parent 1': ' Hi!Administrator:Hi there.Hello.I\'m good.Sorry I\'m late.I can\'t figure this out, so-Administrator:You\'re actually on time, so-Oh, good!Administrator:That\'s perfect.Hello.Hi.No, it\'s Parent 1.It\'s under my husband\'s name and I couldn\'t figure out how to change it on my.I\'m not as savvy as my teenage son on computers.So I have two children.Two boys, one 13 and one eight.And a little dog that\'s sitting on my lap.Hi! So, that\'s it.I can go next.So what was the question. It was.So they never, definitely my eight-year-old, never used technology for school.They don\'t even have homework at that age.My 13-year-old, he also has ADD, ADHD and he has an IEP and all that stuff.And so, he would do his homework at school.So I don\'t even know what he\'d used in terms of technology, but he didn\'t use it at home.That\'s for sure.And then afterwards, he had to use it for.It\'s the same situation with the two screens, and he would have his Chromebook open doing school.And the

In [ ]:
list(res.keys())

['Parent 1', 'Parent 2', 'Parent 3', 'Parent 4']

In [ ]:
list(res.values())

[' Hi!Administrator:Hi there.Hello.I\'m good.Sorry I\'m late.I can\'t figure this out, so-Administrator:You\'re actually on time, so-Oh, good!Administrator:That\'s perfect.Hello.Hi.No, it\'s Parent 1.It\'s under my husband\'s name and I couldn\'t figure out how to change it on my.I\'m not as savvy as my teenage son on computers.So I have two children.Two boys, one 13 and one eight.And a little dog that\'s sitting on my lap.Hi! So, that\'s it.I can go next.So what was the question. It was.So they never, definitely my eight-year-old, never used technology for school.They don\'t even have homework at that age.My 13-year-old, he also has ADD, ADHD and he has an IEP and all that stuff.And so, he would do his homework at school.So I don\'t even know what he\'d used in terms of technology, but he didn\'t use it at home.That\'s for sure.And then afterwards, he had to use it for.It\'s the same situation with the two screens, and he would have his Chromebook open doing school.And then I can\'t h

In [ ]:
??re.sub

In [ ]:
??re.subn

In [ ]:
text = textract.process('./ML-for-Good-Hackathon/Data/FocusGroups/Media_Group2.docx')
text = text.decode('utf-8')
text =text.replace('\n',"")
text = text.replace('\t',"")
text = text.replace('(silence)',"")
text = text.replace('. ','.')
split_sent=re.split(r'[?.]', text)
split_sent = list(filter(('').__ne__, split_sent))
dial_characters = ('Administrator','Moderator','Parent','Speaker',' Administrator',' Moderator',' Parent',' Speaker')

In [ ]:
text

'Parent 1: Hello, good afternoon.Parent 2:Hi.Moderator 1:Hi Parent 2 and Parent 1 Thank you guys for joining.Parent 2:Hi.Good afternoon.Parent 1:Good afternoon.Moderator 1:So I think we\'re going to be a group of either five or six parents today so I\'m just going to wait for a few more people to sign on.Parent 1:Okay.Parent 2:Okay. Moderator 1:Hi, Parent 3.Welcome.Parent 3:Hi.Sorry.I had the volume all the way down so I couldn\'t hear but I-Moderator 1:No problem.We\'re just waiting a few minutes for everyone else to join but we\'ll hopefully [inaudible 00:07:46] soon.Parent 3:As soon as I came into my room, my husband\'s out there but I can hear my kids start fighting.He was like, "Why do you start fighting the instant your mother goes into the other room?" Better him than me, I suppose.Moderator 1:It\'s nice they\'re doing something else.Parent 3:Right.Moderator 1:Hi, Parent 6.Welcome.Hi, everyone.Good afternoon.Hi, Parent 4.Welcome.Parent 4:[inaudible 00:08:43].Hi.Moderator 1:Hi, e

In [ ]:
split_sent

['Parent 1: Hello, good afternoon',
 'Parent 2:Hi',
 'Moderator 1:Hi Parent 2 and Parent 1 Thank you guys for joining',
 'Parent 2:Hi',
 'Good afternoon',
 'Parent 1:Good afternoon',
 "Moderator 1:So I think we're going to be a group of either five or six parents today so I'm just going to wait for a few more people to sign on",
 'Parent 1:Okay',
 'Parent 2:Okay',
 ' Moderator 1:Hi, Parent 3',
 'Welcome',
 'Parent 3:Hi',
 'Sorry',
 "I had the volume all the way down so I couldn't hear but I-Moderator 1:No problem",
 "We're just waiting a few minutes for everyone else to join but we'll hopefully [inaudible 00:07:46] soon",
 "Parent 3:As soon as I came into my room, my husband's out there but I can hear my kids start fighting",
 'He was like, "Why do you start fighting the instant your mother goes into the other room',
 '" Better him than me, I suppose',
 "Moderator 1:It's nice they're doing something else",
 'Parent 3:Right',
 'Moderator 1:Hi, Parent 6',
 'Welcome',
 'Hi, everyone',
 'G

In [ ]:
dial_start=[ind for ind,l in enumerate(split_sent) if l.startswith(dial_characters)]

In [ ]:
dial_start

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 9,
 11,
 15,
 18,
 19,
 20,
 26,
 28,
 30,
 31,
 32,
 33,
 46,
 47,
 49,
 55,
 58,
 59,
 66,
 70,
 83,
 85,
 94,
 99,
 100,
 101,
 108,
 113,
 115,
 123,
 136,
 140,
 142,
 144,
 145,
 147,
 151,
 153,
 166,
 174,
 176,
 177,
 179,
 181,
 184,
 188,
 189,
 194,
 195,
 198,
 201,
 202,
 203,
 205,
 209,
 220,
 221,
 223,
 224,
 230,
 232,
 241,
 243,
 246,
 249,
 265,
 268,
 273,
 280,
 284,
 286,
 288,
 290,
 291,
 294,
 295,
 297,
 300,
 301,
 303,
 324,
 329,
 331,
 332,
 381,
 384,
 392,
 398,
 399,
 400,
 401,
 407,
 427,
 431,
 439,
 441,
 448,
 449,
 450,
 452,
 456,
 459,
 462,
 480,
 481,
 483,
 487,
 488,
 505,
 510,
 512,
 524,
 527,
 528,
 529,
 557,
 559,
 560,
 561,
 562,
 571,
 574,
 588,
 589,
 592,
 595,
 597,
 600,
 609,
 610,
 611,
 629,
 631,
 632,
 633,
 672,
 674,
 676,
 679,
 712,
 720,
 726,
 730,
 731,
 732,
 742,
 744,
 746,
 748]

In [ ]:
list_of_dial = []
for i in range(len(dial_start)-1):
  list_of_dial.append('.'.join(split_sent[dial_start[i]:dial_start[i+1]]))
list_of_Parent_dial = [sent for sent in list_of_dial if sent.startswith('Parent')]
num_ind = [sent.split(" ")[1].split(':')[0] for sent in list_of_Parent_dial]
parent_num = list(set([int(word) for sent in num_ind for word in sent.split() if word.isdigit()]))
all_dial_comb = []
for i in parent_num:
      all_dial_comb.append('.'.join([sent for sent in list_of_Parent_dial if sent.startswith(('Parent '+str(i)+':',' Parent '+str(i)+':'))]))
all_d_comb_n=[]
for i in range(len(all_dial_comb)):
  all_d_comb_n.append(all_dial_comb[i].replace('Parent '+str(i+1)+":","").replace(' Parent '+str(i+1)+":",""))

In [ ]:
list_of_dial

['Parent 1: Hello, good afternoon',
 'Parent 2:Hi',
 'Moderator 1:Hi Parent 2 and Parent 1 Thank you guys for joining',
 'Parent 2:Hi.Good afternoon',
 'Parent 1:Good afternoon',
 "Moderator 1:So I think we're going to be a group of either five or six parents today so I'm just going to wait for a few more people to sign on",
 'Parent 1:Okay',
 'Parent 2:Okay',
 ' Moderator 1:Hi, Parent 3.Welcome',
 "Parent 3:Hi.Sorry.I had the volume all the way down so I couldn't hear but I-Moderator 1:No problem.We're just waiting a few minutes for everyone else to join but we'll hopefully [inaudible 00:07:46] soon",
 'Parent 3:As soon as I came into my room, my husband\'s out there but I can hear my kids start fighting.He was like, "Why do you start fighting the instant your mother goes into the other room." Better him than me, I suppose',
 "Moderator 1:It's nice they're doing something else",
 'Parent 3:Right',
 'Moderator 1:Hi, Parent 6.Welcome.Hi, everyone.Good afternoon.Hi, Parent 4.Welcome',
 '

In [ ]:
list_of_Parent_dial

['Parent 1: Hello, good afternoon',
 'Parent 2:Hi',
 'Parent 2:Hi.Good afternoon',
 'Parent 1:Good afternoon',
 'Parent 1:Okay',
 'Parent 2:Okay',
 "Parent 3:Hi.Sorry.I had the volume all the way down so I couldn't hear but I-Moderator 1:No problem.We're just waiting a few minutes for everyone else to join but we'll hopefully [inaudible 00:07:46] soon",
 'Parent 3:As soon as I came into my room, my husband\'s out there but I can hear my kids start fighting.He was like, "Why do you start fighting the instant your mother goes into the other room." Better him than me, I suppose',
 'Parent 3:Right',
 'Parent 4:[inaudible 00:08:43].Hi',
 'Parent 5:Yes, yes',
 'Parent 5:[crosstalk 00:08:56]',
 "Parent 2:I'm Parent 2.I live in Brooklyn.I'm an academic advisor.I have a 14-year-old son.And a movie that I just saw was Thunderforce, which is cool with two women that are like superheroes.That was funny",
 "Parent 5:Hi.My name is Parent 5.I'm Andrew's mom.He's 12 years old and we live in Brooklyn a